<div style="text-align: right"><b> Ana Valentina López Chacón & Camila Palomeque </b></div>
<div style="text-align: right"><b> Herramientas y Aplicaciones de la Inteligencia Artificial, 2024 - 2025 </b></div>

## **Proyecto: Detección de Argumentos**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# path = '/content/drive/MyDrive/Colab Notebooks/HAIA/LAB3/'

Mounted at /content/drive


In [2]:
!pip install pysentimiento transformers datasets accelerate evaluate --quiet
!pip install ipdb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd
import re

In [9]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\w\s\(\),.]', '', text)
    return text

In [17]:
def classify_relationship(arg1, arg2):
    arg1 = preprocess_text(arg1)
    arg2 = preprocess_text(arg2)
    prompt = f"""Goal: {arg1}
                Argument: {arg2}
                Question: Is the argument infered, in conflict or is a rephrase of the goal? Answer with only 'Inference', 'Conflict' or 'Rephrase'."""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=10)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    return answer

In [22]:
a = classify_relationship('End poverty in all its forms everywhere',
                      'By 2030, 590 million people may still live in extreme poverty if current trends persist.')
a[0].upper() + a[1:]

'Inference'

### **Relación de Cada Goal con sus Argumentos**

In [23]:
import os
import json

def combine_arguments_as_lists(directory_path):
    list_of_argument_lists = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                arguments = data.get("arguments", [])
                list_of_argument_lists.append(arguments)

    return list_of_argument_lists

directory = '/content/drive/MyDrive/Colab Notebooks/HAIA/PROYECTO/results_poligpt'
all_arguments = combine_arguments_as_lists(directory)

for idx, argument_list in enumerate(all_arguments):
    print(f"Goal {idx+1}: {len(argument_list)}")

Goal 1: 9
Goal 2: 9
Goal 3: 19
Goal 4: 9
Goal 5: 16
Goal 6: 11
Goal 7: 7
Goal 8: 10
Goal 9: 8
Goal 10: 12
Goal 11: 15
Goal 12: 14
Goal 13: 14
Goal 14: 13
Goal 15: 11
Goal 16: 9
Goal 17: 10


In [24]:
all_goals = ["End poverty in all its forms everywhere",
             "End hunger, achieve food security and improved nutrition and promote sustainable agriculture",
             "Ensure healthy lives and promote well-being for all at all ages",
             "Ensure inclusive and equitable quality education",
             "Achieve gender equality and empower all women and girls",
             "Ensure availability and sustainable management of water and sanitation for all",
             "Ensure access to affordable, reliable, sustainable and modern energy for all",
             "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all",
             "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation",
             "Reduce inequality within and among countries",
             "Make cities and human settlements inclusive, safe, resilient and sustainable",
             "Ensure sustainable consumption and production patterns",
             "Take urgent action to combat climate change and its impacts",
             "Conserve and sustainably use the oceans, seas and marine resources for sustainable development",
             "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss",
             "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels",
             "Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development"]

len(all_goals)

17

In [26]:
for idx, argument_list in enumerate(all_arguments):
  relationships = []
  goal = all_goals[idx]
  print(f"Goal {idx+1}: {goal}")
  for i in range(len(argument_list)):
    arg = argument_list[i]

    relationship = classify_relationship(goal, arg)

    if relationship != 3:
      relationships.append({
            'id_argumento1': goal,
            'id_argumento2': arg,
            'relacion': relationship[0].upper() + relationship[1:]
        })

  df = pd.DataFrame(relationships)
  csv_path = f'/content/drive/MyDrive/Colab Notebooks/HAIA/PROYECTO/relationships_flant5_goal_arg/args_poligpt_goal_{idx+1}.csv'
  df.to_csv(csv_path, index=False)

Goal 1: End poverty in all its forms everywhere
Goal 2: End hunger, achieve food security and improved nutrition and promote sustainable agriculture
Goal 3: Ensure healthy lives and promote well-being for all at all ages
Goal 4: Ensure inclusive and equitable quality education
Goal 5: Achieve gender equality and empower all women and girls
Goal 6: Ensure availability and sustainable management of water and sanitation for all
Goal 7: Ensure access to affordable, reliable, sustainable and modern energy for all
Goal 8: Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all
Goal 9: Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation
Goal 10: Reduce inequality within and among countries
Goal 11: Make cities and human settlements inclusive, safe, resilient and sustainable
Goal 12: Ensure sustainable consumption and production patterns
Goal 13: Take urgent action to combat clima

### **Relación de Argumentos por Goal**

In [ ]:
import os
import json

def combine_arguments_as_lists(directory_path):
    list_of_argument_lists = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                arguments = data.get("arguments", [])
                list_of_argument_lists.append(arguments)

    return list_of_argument_lists

directory = '/content/drive/MyDrive/Colab Notebooks/HAIA/PROYECTO/results_poligpt'
all_arguments = combine_arguments_as_lists(directory)

for idx, argument_list in enumerate(all_arguments):
    print(f"Goal {idx+1}: {len(argument_list)}")

Goal 1: 9
Goal 2: 9
Goal 3: 19
Goal 4: 9
Goal 5: 16
Goal 6: 11
Goal 7: 7
Goal 8: 10
Goal 9: 8
Goal 10: 12
Goal 11: 15
Goal 12: 14
Goal 13: 14
Goal 14: 13
Goal 15: 11
Goal 16: 9
Goal 17: 10


In [ ]:

for idx, argument_list in enumerate(all_arguments):
  relationships = []
  print(f"Goal {idx+1}")
  for i in range(len(argument_list)):
    for j in range(i+1, len(argument_list)):
      from_arg_text = argument_list[i]
      to_arg_text = argument_list[j]

      relationship = classify_relationship(from_arg_text, to_arg_text)

      relationship_map = {
        0: "Inference",
        1: "Conflict",
        2: "Rephrase"
      }

      if relationship != 3:
        relationships.append({
              'id_argumento1': from_arg_text,
              'id_argumento2': to_arg_text,
              'relacion': relationship_map[relationship]
          })

  df = pd.DataFrame(relationships)
  csv_path = f'/content/drive/MyDrive/Colab Notebooks/HAIA/PROYECTO/relationships_per_goal/arg_poligpt_goal_{idx+1}.csv'
  df.to_csv(csv_path, index=False)


Goal 1
Goal 2
Goal 3
Goal 4
Goal 5
Goal 6
Goal 7
Goal 8
Goal 9
Goal 10
Goal 11
Goal 12
Goal 13
Goal 14
Goal 15
Goal 16
Goal 17


### **Relación de Argumentos General**

In [ ]:
import os
import json

def combine_arguments(directory_path):
    combined_arguments = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                arguments = data.get("arguments", [])
                combined_arguments.extend(arguments)

    return combined_arguments

# Example usage:
directory = '/content/drive/MyDrive/Colab Notebooks/HAIA/PROYECTO/results_poligpt'
all_arguments = combine_arguments(directory)

print(f"Total arguments collected: {len(all_arguments)}")

Total arguments collected: 196


In [ ]:
relationships_total = []

for i in range(len(all_arguments)):
  print(f'Arg {i}')
  for j in range(i+1, len(all_arguments)):
    from_arg_text = all_arguments[i]
    to_arg_text = all_arguments[j]

    relationship = classify_relationship(from_arg_text, to_arg_text)

    relationship_map = {
      0: "Inference",
      1: "Conflict",
      2: "Rephrase"
    }

    if relationship != 3:
      relationships_total.append({
            'id_argumento1': from_arg_text,
            'id_argumento2': to_arg_text,
            'relacion': relationship_map[relationship]
        })

df_global = pd.DataFrame(relationships_total)
df_global

Arg 0
Arg 1
Arg 2
Arg 3
Arg 4
Arg 5
Arg 6
Arg 7
Arg 8
Arg 9
Arg 10
Arg 11
Arg 12
Arg 13
Arg 14
Arg 15
Arg 16
Arg 17
Arg 18
Arg 19
Arg 20
Arg 21
Arg 22
Arg 23
Arg 24
Arg 25
Arg 26
Arg 27
Arg 28
Arg 29
Arg 30
Arg 31
Arg 32
Arg 33
Arg 34
Arg 35
Arg 36
Arg 37
Arg 38
Arg 39
Arg 40
Arg 41
Arg 42
Arg 43
Arg 44
Arg 45
Arg 46
Arg 47
Arg 48
Arg 49
Arg 50
Arg 51
Arg 52
Arg 53
Arg 54
Arg 55
Arg 56
Arg 57
Arg 58
Arg 59
Arg 60
Arg 61
Arg 62
Arg 63
Arg 64
Arg 65
Arg 66
Arg 67
Arg 68
Arg 69
Arg 70
Arg 71
Arg 72
Arg 73
Arg 74
Arg 75
Arg 76
Arg 77
Arg 78
Arg 79
Arg 80
Arg 81
Arg 82
Arg 83
Arg 84
Arg 85
Arg 86
Arg 87
Arg 88
Arg 89
Arg 90
Arg 91
Arg 92
Arg 93
Arg 94
Arg 95
Arg 96
Arg 97
Arg 98
Arg 99
Arg 100
Arg 101
Arg 102
Arg 103
Arg 104
Arg 105
Arg 106
Arg 107
Arg 108
Arg 109
Arg 110
Arg 111
Arg 112
Arg 113
Arg 114
Arg 115
Arg 116
Arg 117
Arg 118
Arg 119
Arg 120
Arg 121
Arg 122
Arg 123
Arg 124
Arg 125
Arg 126
Arg 127
Arg 128
Arg 129
Arg 130
Arg 131
Arg 132
Arg 133
Arg 134
Arg 135
Arg 136
Arg 137
Arg 13

,id_argumento1,id_argumento2,relacion
0,The COVID-19 pandemic and subsequent shocks fr...,Ending poverty requires a wide-ranging approac...,Conflict
1,The COVID-19 pandemic and subsequent shocks fr...,Despite increasing efforts and commitments to ...,Conflict
2,The COVID-19 pandemic and subsequent shocks fr...,Without a substantial acceleration in poverty ...,Conflict
3,The COVID-19 pandemic and subsequent shocks fr...,Working poverty disproportionately affects som...,Conflict
4,The COVID-19 pandemic and subsequent shocks fr...,Achieving universal coverage of social protect...,Conflict
...,...,...,...
18333,The trillions needed to achieve the SDGs requi...,Rising borrowing costs put developing countrie...,Conflict
18334,The trillions needed to achieve the SDGs requi...,"High-quality, timely and disaggregated data ar...",Conflict
18335,Rising borrowing costs put developing countrie...,"High-quality, timely and disaggregated data ar...",Inference
18336,Rising borrowing costs put developing countrie...,"Globally, one in three people are still offlin...",Conflict


In [ ]:
csv_path = '/content/drive/MyDrive/Colab Notebooks/HAIA/PROYECTO/argument_relationships_todos_con_todos.csv'
df_global.to_csv(csv_path, index=False)

In [ ]:
df_global['relacion'].value_counts()

,count
relacion,
Inference,13519
Conflict,4608
Rephrase,211
